# Estimate translational and Coriolis velocities

In [1]:
%matplotlib notebook

In [2]:
import numpy as np

In [3]:
import matplotlib.pyplot as plt

In [4]:
import pandas as pd

First we read/define some parameters (see [Parameters.ipynb](./Parameters.ipynb))

In [5]:
from parameters import *

In [6]:
radcols=['64ne', '64se', '64sw', '64nw', '50ne', '50se', '50sw', '50nw',
       '34ne', '34se', '34sw', '34nw']

## Holland Parameter Evaluation

The input data comes from the bulletins and are saved in txt format during the step 1.

In [7]:
path='test/'

In [8]:
filename='step1.txt'

In [9]:
fk=0.92  # coefficient for going from 1m to 10m in velocities ????????????????????????

Reading the data into a pandas DataFrame 

In [12]:
inpdat = pd.read_csv(path+filename, delimiter='\t')

In [13]:
inpdat.head()

,time,t,hurName,lat,lon,pcenter,penv,rmax,vmax,34ne,...,34sw,50ne,50nw,50se,50sw,64ne,64nw,64se,64sw,dp
0,0.0,2016-09-29 12:00:00,MATTHEW,14.2,-65.5,995,-99,68524.0,30.866667,279652.0,...,218536.0,135196.0,0.0,0.0,127788.0,0.0,0.0,0.0,0.0,-109400
1,1.0,2016-09-29 13:00:00,MATTHEW,14.2,-65.7,998,-99,46300.0,29.323333,288912.0,...,187052.0,122232.0,0.0,64820.0,103712.0,0.0,0.0,0.0,0.0,-109700
2,2.0,2016-09-29 14:00:00,MATTHEW,14.3,-65.9,997,-99,57412.0,33.953333,262984.0,...,211128.0,114824.0,77784.0,0.0,101860.0,64820.0,0.0,0.0,0.0,-109600
3,3.0,2016-09-29 15:00:00,MATTHEW,14.3,-66.1,995,-99,48152.0,33.438889,253724.0,...,240760.0,112972.0,0.0,51856.0,109268.0,0.0,0.0,0.0,53708.0,-109400
4,4.0,2016-09-29 16:00:00,MATTHEW,14.3,-66.3,993,-99,42596.0,33.953333,266688.0,...,296320.0,122232.0,0.0,61116.0,146308.0,0.0,0.0,50004.0,70376.0,-109200


Check if we cross the International Date Line

In [14]:
if inpdat.lon.apply(np.sign).diff().sum() > 0:
    m=inpdat.lon != inpdat.lon[0]
    inpdat.lon[m]+360. if inpdat.lon[0] > 0 else npdat.lon[m]-360.

## Calculate translation velocity

In [15]:
x=inpdat.lon
y=inpdat.lat

In [16]:
dt=np.gradient(inpdat.time)*3600 # compute dt (translate time from hours to sec)

In [17]:
dx_dt = np.gradient(x,dt)
dy_dt = np.gradient(y,dt)
velocity = np.array([ [dx_dt[i], dy_dt[i]] for i in range(dx_dt.size)])

In [18]:
#velocity

In [19]:
vtrx = velocity[:,0] * deg2m * np.cos(np.radians(inpdat.lat.values))  #adjust for latitude
vtry = velocity[:,1] * deg2m

In [20]:
vtr = np.sqrt(vtrx**2+vtry**2)

In [21]:
#print vtrx,vtry,vtr

Compute the tangent of unit vector value, see http://stackoverflow.com/questions/28269379/curve-curvature-in-numpy

In [22]:
ds_dt = np.sqrt(dx_dt * dx_dt + dy_dt * dy_dt)

In [23]:
tangent = np.array([1/ds_dt] * 2).transpose() * velocity

In [24]:
phi=np.arctan2(tangent[:,1],tangent[:,0]) # the angle of the velocity vector

In [25]:
cosfi = np.cos(phi)
sinfi = np.sin(phi)

In [26]:
# extend dataset to save new data
inpdat['vtrx']=vtrx
inpdat['vtry']=vtry
inpdat['vtr']=vtr
inpdat['cosfi']=cosfi
inpdat['sinfi']=sinfi

In [27]:
inpdat.head()

,time,t,hurName,lat,lon,pcenter,penv,rmax,vmax,34ne,...,64ne,64nw,64se,64sw,dp,vtrx,vtry,vtr,cosfi,sinfi
0,0.0,2016-09-29 12:00:00,MATTHEW,14.2,-65.5,995,-99,68524.0,30.866667,279652.0,...,0.0,0.0,0.0,0.0,-109400,-5.995689,0.000000,5.995689,-1.000000,1.224647e-16
1,1.0,2016-09-29 13:00:00,MATTHEW,14.2,-65.7,998,-99,46300.0,29.323333,288912.0,...,0.0,0.0,0.0,0.0,-109700,-5.995689,1.546165,6.191843,-0.970143,2.425356e-01
2,2.0,2016-09-29 14:00:00,MATTHEW,14.3,-65.9,997,-99,57412.0,33.953333,262984.0,...,64820.0,0.0,0.0,0.0,-109600,-5.993032,1.546165,6.189270,-0.970143,2.425356e-01
3,3.0,2016-09-29 15:00:00,MATTHEW,14.3,-66.1,995,-99,48152.0,33.438889,253724.0,...,0.0,0.0,0.0,53708.0,-109400,-5.993032,0.000000,5.993032,-1.000000,1.224647e-16
4,4.0,2016-09-29 16:00:00,MATTHEW,14.3,-66.3,993,-99,42596.0,33.953333,266688.0,...,0.0,0.0,50004.0,70376.0,-109200,-4.494774,0.000000,4.494774,-1.000000,1.224647e-16


In [28]:
cols=['w'+ x for x in radcols]

In [29]:
#temp = np.zeros((time.size, 12))
#d = pd.DataFrame(temp, columns = cols)

In [30]:
inpdat

,time,t,hurName,lat,lon,pcenter,penv,rmax,vmax,34ne,...,64ne,64nw,64se,64sw,dp,vtrx,vtry,vtr,cosfi,sinfi
0,0.0,2016-09-29 12:00:00,MATTHEW,14.2,-65.5,995,-99,68524.0,30.866667,279652.0,...,0.0,0.0,0.0,0.0,-109400,-5.995689,0.000000,5.995689,-1.000000e+00,1.224647e-16
1,1.0,2016-09-29 13:00:00,MATTHEW,14.2,-65.7,998,-99,46300.0,29.323333,288912.0,...,0.0,0.0,0.0,0.0,-109700,-5.995689,1.546165,6.191843,-9.701425e-01,2.425356e-01
2,2.0,2016-09-29 14:00:00,MATTHEW,14.3,-65.9,997,-99,57412.0,33.953333,262984.0,...,64820.0,0.0,0.0,0.0,-109600,-5.993032,1.546165,6.189270,-9.701425e-01,2.425356e-01
3,3.0,2016-09-29 15:00:00,MATTHEW,14.3,-66.1,995,-99,48152.0,33.438889,253724.0,...,0.0,0.0,0.0,53708.0,-109400,-5.993032,0.000000,5.993032,-1.000000e+00,1.224647e-16
4,4.0,2016-09-29 16:00:00,MATTHEW,14.3,-66.3,993,-99,42596.0,33.953333,266688.0,...,0.0,0.0,50004.0,70376.0,-109200,-4.494774,0.000000,4.494774,-1.000000e+00,1.224647e-16
5,5.0,2016-09-29 17:00:00,MATTHEW,14.3,-66.4,992,-99,68524.0,31.381111,253724.0,...,0.0,0.0,0.0,0.0,-109100,-4.494774,1.546165,4.753275,-9.486833e-01,3.162278e-01
6,6.0,2016-09-29 18:00:00,MATTHEW,14.4,-66.6,990,-99,72228.0,31.381111,251872.0,...,0.0,0.0,0.0,0.0,-108900,-7.487946,1.546165,7.645912,-9.805807e-01,1.961161e-01
7,7.0,2016-09-29 19:00:00,MATTHEW,14.4,-66.9,990,-99,51856.0,31.381111,257428.0,...,0.0,0.0,0.0,0.0,-108900,-7.487946,1.546165,7.645912,-9.805807e-01,1.961161e-01
8,8.0,2016-09-29 20:00:00,MATTHEW,14.5,-67.1,989,-99,53708.0,32.410000,250020.0,...,0.0,0.0,0.0,0.0,-108800,-5.987664,3.092330,6.739037,-8.944272e-01,4.472136e-01
9,9.0,2016-09-29 21:00:00,MATTHEW,14.6,-67.3,989,-99,44448.0,32.924444,238908.0,...,0.0,0.0,0.0,46300.0,-108800,-7.481190,0.000000,7.481190,-1.000000e+00,1.224647e-16


In [31]:
an=np.array([tetaNE, tetaSE, tetaSW, tetaNW,tetaNE, tetaSE, tetaSW, tetaNW,tetaNE, tetaSE, tetaSW, tetaNW])# to be used
sinan = np.sin(np.radians(an+90))  # an +90 = angle of tangential wind
cosan=np.cos(np.radians(an+90))

In [32]:
V0=np.array([64, 64, 64, 64, 50, 50, 50, 50, 34, 34, 34, 34])*kt2ms*fk #translate knots to m/s and from 1km to 10km

In [33]:
R=inpdat.ix[:,radcols].copy()

In [34]:
R=R[R>0]

In [35]:
RATIO = (rmax0/R)**b0    # assume exponential decay eqs (13) from JRC report
EXPRATIO = np.exp(-RATIO)  #                       "

In [36]:
RATIO

,64ne,64se,64sw,64nw,50ne,50se,50sw,50nw,34ne,34se,34sw,34nw
0,NaN,NaN,NaN,NaN,0.100944,NaN,0.108006,NaN,0.042198,0.148230,0.056729,0.043938
1,NaN,NaN,NaN,NaN,0.113924,0.243885,0.138753,NaN,0.040580,0.151657,0.068372,0.041214
2,0.243885,NaN,NaN,NaN,0.122800,NaN,0.141785,0.195960,0.045428,0.158965,0.059126,0.047843
3,NaN,NaN,0.305625,NaN,0.125219,0.318770,0.130330,NaN,0.047424,0.190504,0.050505,0.042536
4,NaN,0.332989,0.220967,NaN,0.113924,0.261728,0.091815,NaN,0.044672,0.185321,0.039366,0.044672
5,NaN,NaN,NaN,NaN,0.155232,0.228152,0.093229,0.318770,0.047424,0.034639,0.033727,0.043938
6,NaN,NaN,NaN,NaN,0.138753,0.220967,0.093229,0.228152,0.047843,0.180390,0.044672,0.045047
7,NaN,NaN,NaN,NaN,0.116030,0.365207,0.108006,NaN,0.046607,0.138753,0.040580,0.054505
8,NaN,NaN,NaN,NaN,0.111886,0.318770,0.125219,0.228152,0.048269,0.166947,0.043227,0.061716
9,NaN,NaN,0.365207,NaN,0.130330,0.261728,0.118209,0.220967,0.050975,0.201709,0.040580,0.100944


In [37]:
VT=vtr[:,np.newaxis]*(cosfi[:,np.newaxis] * cosan + sinfi[:,np.newaxis] * sinan)*(1-EXPRATIO)   # Eq (15) from JRC report

In [38]:
VT

,64ne,64se,64sw,64nw,50ne,50se,50sw,50nw,34ne,34se,34sw,34nw
0,NaN,NaN,NaN,NaN,0.407069,NaN,-0.434040,NaN,0.175181,-5.840765e-01,-0.233815,1.822474e-01
1,NaN,NaN,NaN,NaN,0.571691,-6.894515e-01,-0.687876,NaN,0.211147,-4.482793e-01,-0.350889,1.286249e-01
2,1.148608,NaN,NaN,NaN,0.613301,NaN,-0.701580,5.666715e-01,0.235702,-4.680177e-01,-0.304702,1.487626e-01
3,NaN,NaN,-1.115948,NaN,0.498764,-1.156714e+00,-0.517824,NaN,0.196280,-7.350643e-01,-0.208711,1.764758e-01
4,NaN,-9.001608e-01,-0.630116,NaN,0.342219,-7.318943e-01,-0.278819,NaN,0.138855,-5.376454e-01,-0.122686,1.388547e-01
5,NaN,NaN,NaN,NaN,0.611291,-4.336422e-01,-0.378446,5.802329e-01,0.196917,-7.237203e-02,-0.140999,9.137874e-02
6,NaN,NaN,NaN,NaN,0.824211,-8.408432e-01,-0.566298,8.651895e-01,0.297202,-7.000305e-01,-0.277937,1.868126e-01
7,NaN,NaN,NaN,NaN,0.696944,-1.297578e+00,-0.651305,NaN,0.289699,-5.494743e-01,-0.252996,2.249784e-01
8,NaN,NaN,NaN,NaN,0.676748,-5.816909e-01,-0.752459,4.347319e-01,0.301263,-3.276630e-01,-0.270470,1.275443e-01
9,NaN,NaN,-1.618459,NaN,0.646408,-1.218179e+00,-0.589781,1.048777e+00,0.262901,-9.663116e-01,-0.210373,5.079252e-01


In [39]:
VT.loc[inpdat.lat<0] = -VT # reverse for south hemishpere

In [40]:
VT

,64ne,64se,64sw,64nw,50ne,50se,50sw,50nw,34ne,34se,34sw,34nw
0,NaN,NaN,NaN,NaN,0.407069,NaN,-0.434040,NaN,0.175181,-5.840765e-01,-0.233815,1.822474e-01
1,NaN,NaN,NaN,NaN,0.571691,-6.894515e-01,-0.687876,NaN,0.211147,-4.482793e-01,-0.350889,1.286249e-01
2,1.148608,NaN,NaN,NaN,0.613301,NaN,-0.701580,5.666715e-01,0.235702,-4.680177e-01,-0.304702,1.487626e-01
3,NaN,NaN,-1.115948,NaN,0.498764,-1.156714e+00,-0.517824,NaN,0.196280,-7.350643e-01,-0.208711,1.764758e-01
4,NaN,-9.001608e-01,-0.630116,NaN,0.342219,-7.318943e-01,-0.278819,NaN,0.138855,-5.376454e-01,-0.122686,1.388547e-01
5,NaN,NaN,NaN,NaN,0.611291,-4.336422e-01,-0.378446,5.802329e-01,0.196917,-7.237203e-02,-0.140999,9.137874e-02
6,NaN,NaN,NaN,NaN,0.824211,-8.408432e-01,-0.566298,8.651895e-01,0.297202,-7.000305e-01,-0.277937,1.868126e-01
7,NaN,NaN,NaN,NaN,0.696944,-1.297578e+00,-0.651305,NaN,0.289699,-5.494743e-01,-0.252996,2.249784e-01
8,NaN,NaN,NaN,NaN,0.676748,-5.816909e-01,-0.752459,4.347319e-01,0.301263,-3.276630e-01,-0.270470,1.275443e-01
9,NaN,NaN,-1.618459,NaN,0.646408,-1.218179e+00,-0.589781,1.048777e+00,0.262901,-9.663116e-01,-0.210373,5.079252e-01


In [41]:
VV = V0-VT   # substract translational velocity from TC velocity

In [42]:
deltalatWR=R/deg2m*np.sin(np.radians(an))

In [43]:
deltalatWR

,64ne,64se,64sw,64nw,50ne,50se,50sw,50nw,34ne,34se,34sw,34nw
0,NaN,NaN,NaN,NaN,0.858738,NaN,-0.811684,NaN,1.776293,-0.623467,-1.388097,1.717475
1,NaN,NaN,NaN,NaN,0.776393,-0.411724,-0.658758,NaN,1.835111,-0.611704,-1.188117,1.811584
2,0.411724,NaN,NaN,NaN,0.729339,NaN,-0.646994,0.494068,1.670421,-0.588177,-1.341042,1.599840
3,NaN,NaN,-0.341142,NaN,0.717575,-0.329379,-0.694048,NaN,1.611604,-0.505832,-1.529259,1.764530
4,NaN,-0.317615,-0.447014,NaN,0.776393,-0.388197,-0.929319,NaN,1.693948,-0.517595,-1.882165,1.693948
5,NaN,NaN,NaN,NaN,0.599940,-0.435251,-0.917555,0.329379,1.611604,-2.093908,-2.140963,1.717475
6,NaN,NaN,NaN,NaN,0.658758,-0.447014,-0.917555,0.435251,1.599840,-0.529359,-1.693948,1.682185
7,NaN,NaN,NaN,NaN,0.764629,-0.294088,-0.811684,NaN,1.635131,-0.658758,-1.835111,1.435151
8,NaN,NaN,NaN,NaN,0.788157,-0.329379,-0.717575,0.435251,1.588077,-0.564649,-1.741003,1.293988
9,NaN,NaN,-0.294088,NaN,0.694048,-0.388197,-0.752866,0.447014,1.517495,-0.482305,-1.835111,0.858738


In [44]:
latWR=inpdat.lat[:,np.newaxis]+deltalatWR

In [45]:
latWR

,64ne,64se,64sw,64nw,50ne,50se,50sw,50nw,34ne,34se,34sw,34nw
0,NaN,NaN,NaN,NaN,15.058738,NaN,13.388316,NaN,15.976293,13.576533,12.811903,15.917475
1,NaN,NaN,NaN,NaN,14.976393,13.788276,13.541242,NaN,16.035111,13.588296,13.011883,16.011584
2,14.711724,NaN,NaN,NaN,15.029339,NaN,13.653006,14.794068,15.970421,13.711823,12.958958,15.899840
3,NaN,NaN,13.958858,NaN,15.017575,13.970621,13.605952,NaN,15.911604,13.794168,12.770741,16.064530
4,NaN,13.982385,13.852986,NaN,15.076393,13.911803,13.370681,NaN,15.993948,13.782405,12.417835,15.993948
5,NaN,NaN,NaN,NaN,14.899940,13.864749,13.382445,14.629379,15.911604,12.206092,12.159037,16.017475
6,NaN,NaN,NaN,NaN,15.058758,13.952986,13.482445,14.835251,15.999840,13.870641,12.706052,16.082185
7,NaN,NaN,NaN,NaN,15.164629,14.105912,13.588316,NaN,16.035131,13.741242,12.564889,15.835151
8,NaN,NaN,NaN,NaN,15.288157,14.170621,13.782425,14.935251,16.088077,13.935351,12.758997,15.793988
9,NaN,NaN,14.305912,NaN,15.294048,14.211803,13.847134,15.047014,16.117495,14.117695,12.764889,15.458738


In [46]:
fWR=2*omega*np.abs(np.sin(np.radians(latWR))) # Coriolis parameter f=2*Omega*sin(lat)
Vnco=((VV+R*fWR/2)**2-(R*fWR/2)**2)**0.5

In [47]:
Vnco=Vnco.replace(np.nan,0)

In [48]:
Vnco

,64ne,64se,64sw,64nw,50ne,50se,50sw,50nw,34ne,34se,34sw,34nw
0,0.000000,0.000000,0.000000,0.000000,25.685045,0.000000,26.161934,0.000000,20.773219,18.274868,19.534445,20.608431
1,0.000000,0.000000,0.000000,0.000000,25.285722,25.452586,26.058509,0.000000,20.893198,18.110921,19.263473,20.915893
2,30.315362,0.000000,0.000000,0.000000,25.123333,0.000000,26.055893,24.499624,20.452050,18.085768,19.538575,20.351526
3,0.000000,0.000000,32.334880,0.000000,25.204719,25.715476,25.984405,0.000000,20.334297,18.155295,19.796023,20.766386
4,0.000000,32.057249,32.122624,0.000000,25.529823,25.442486,26.288832,0.000000,20.615410,17.985921,20.326892,20.615410
5,0.000000,0.000000,0.000000,0.000000,24.756404,25.264011,26.362415,24.017778,20.333641,20.618972,20.763431,20.727831
6,0.000000,0.000000,0.000000,0.000000,24.722219,25.709742,26.567329,24.041325,20.222630,18.186833,20.189763,20.559208
7,0.000000,0.000000,0.000000,0.000000,25.153531,25.769557,26.408986,0.000000,20.325786,18.345570,20.409975,19.850360
8,0.000000,0.000000,0.000000,0.000000,25.255164,25.152396,26.301417,24.480355,20.210466,17.908814,20.292461,19.585819
9,0.000000,0.000000,32.730365,0.000000,25.026409,25.950845,26.236194,23.907438,20.080951,18.355665,20.423490,18.014881


In [49]:
#change header
Vnco.columns = cols

In [50]:
# extend dataset to save the velocities
inpdat = pd.concat([inpdat, Vnco], axis=1)

In [51]:
inpdat.head()

,time,t,hurName,lat,lon,pcenter,penv,rmax,vmax,34ne,...,w64sw,w64nw,w50ne,w50se,w50sw,w50nw,w34ne,w34se,w34sw,w34nw
0,0.0,2016-09-29 12:00:00,MATTHEW,14.2,-65.5,995,-99,68524.0,30.866667,279652.0,...,0.000000,0.0,25.685045,0.000000,26.161934,0.000000,20.773219,18.274868,19.534445,20.608431
1,1.0,2016-09-29 13:00:00,MATTHEW,14.2,-65.7,998,-99,46300.0,29.323333,288912.0,...,0.000000,0.0,25.285722,25.452586,26.058509,0.000000,20.893198,18.110921,19.263473,20.915893
2,2.0,2016-09-29 14:00:00,MATTHEW,14.3,-65.9,997,-99,57412.0,33.953333,262984.0,...,0.000000,0.0,25.123333,0.000000,26.055893,24.499624,20.452050,18.085768,19.538575,20.351526
3,3.0,2016-09-29 15:00:00,MATTHEW,14.3,-66.1,995,-99,48152.0,33.438889,253724.0,...,32.334880,0.0,25.204719,25.715476,25.984405,0.000000,20.334297,18.155295,19.796023,20.766386
4,4.0,2016-09-29 16:00:00,MATTHEW,14.3,-66.3,993,-99,42596.0,33.953333,266688.0,...,32.122624,0.0,25.529823,25.442486,26.288832,0.000000,20.615410,17.985921,20.326892,20.615410


In [52]:
vs = inpdat.vmax*fk-vtr

In [53]:
vmax0vt = np.maximum(vs,Vnco.max(axis=1))

In [54]:
inpdat['vmax0vt'] = vmax0vt

In [55]:
inpdat = inpdat.set_index('time')


In [56]:
inpdat.head()

,t,hurName,lat,lon,pcenter,penv,rmax,vmax,34ne,34nw,...,w64nw,w50ne,w50se,w50sw,w50nw,w34ne,w34se,w34sw,w34nw,vmax0vt
time,,,,,,,,,,,,,,,,,,,,,
0.0,2016-09-29 12:00:00,MATTHEW,14.2,-65.5,995,-99,68524.0,30.866667,279652.0,270392.0,...,0.0,25.685045,0.000000,26.161934,0.000000,20.773219,18.274868,19.534445,20.608431,26.161934
1.0,2016-09-29 13:00:00,MATTHEW,14.2,-65.7,998,-99,46300.0,29.323333,288912.0,285208.0,...,0.0,25.285722,25.452586,26.058509,0.000000,20.893198,18.110921,19.263473,20.915893,26.058509
2.0,2016-09-29 14:00:00,MATTHEW,14.3,-65.9,997,-99,57412.0,33.953333,262984.0,251872.0,...,0.0,25.123333,0.000000,26.055893,24.499624,20.452050,18.085768,19.538575,20.351526,30.315362
3.0,2016-09-29 15:00:00,MATTHEW,14.3,-66.1,995,-99,48152.0,33.438889,253724.0,277800.0,...,0.0,25.204719,25.715476,25.984405,0.000000,20.334297,18.155295,19.796023,20.766386,32.334880
4.0,2016-09-29 16:00:00,MATTHEW,14.3,-66.3,993,-99,42596.0,33.953333,266688.0,266688.0,...,0.0,25.529823,25.442486,26.288832,0.000000,20.615410,17.985921,20.326892,20.615410,32.122624


## save output to file

In [57]:
inpdat.to_csv(path+'step2.txt',index=True, sep='\t')

Now we estimating the Rmax. A number of ways apply. See [Estimate Holland Parameters.ipynb](./Estimate Holland Parameters.ipynb)